In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# OpenAI Whisper API를 활용한 음성-텍스트 변환 튜토리얼 (2025년 3월 기준)

OpenAI의 Whisper 모델은 사람의 음성을 높은 정확도로 텍스트로 변환하는 자동 음성 인식(ASR) 모델입니다. 2025년 현재 Whisper는 OpenAI API를 통해 제공되며, 음성 파일을 원어 그대로 **텍스트로 필사(transcription)**하거나 영어로 **번역(translation)**하는 기능을 제공합니다. 이 튜토리얼에서는 Whisper API를 사용하여 음성-텍스트 변환을 구현하는 방법을 단계별로 살펴보겠습니다. 환경 설정부터 기본 사용 방법, 그리고 언어 감지, 타임스탬프 포함 출력, 실시간 음성 처리와 같은 고급 기능까지 다룹니다.

## 1. 환경 설정

Python 코드에서 python-dotenv로 .env 파일을 불러온 뒤, openai.OpenAI() 클래스를 이용해 API 클라이언트 인스턴스를 생성합니다. 이때 API 키는 명시적으로 전달하거나 환경 변수 OPENAI_API_KEY가 설정되어 있어야 합니다. 환경 변수 또는 인자로 API 키를 지정하지 않으면 OpenAI 라이브러리는 다음과 같은 오류를 발생시킵니다:

> OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

아래는 환경 로드 및 클라이언트 생성 예제입니다:


In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os
load_dotenv()
client = OpenAI()

위 코드에서는 .env에서 불러온 API 키로 client 객체를 생성했습니다. 이제 이 client를 통해 DALL-E를 비롯한 OpenAI API 요청을 보낼 수 있습니다. (참고로, api_key를 생략하면 OPENAI_API_KEY 환경 변수를 자동으로 참조합니다.)

## 2. Whisper API 소개 및 기본 음성 → 텍스트 변환

OpenAI Whisper API는 두 가지 주요 **엔드포인트(endpoint)**를 제공합니다:
- Transcriptions 엔드포인트: 제공한 음성 파일을 해당 음성의 원어로 필사(글로 변환) 합니다 (예: 한국어 음성을 입력하면 한국어 텍스트로 출력).
- Translations 엔드포인트: 제공한 음성 파일을 영어로 번역하여 텍스트로 반환합니다.

Whisper API는 현재 Whisper v2 대형 모델(whisper-1로 식별)을 사용하며, MP3, MP4, WAV 등 다양한 음성 파일 형식을 지원합니다.
다만 한 번 요청할 수 있는 파일 크기는 최대 25MB로 제한되어 있으므로 긴 오디오의 경우 분할하여 처리해야 합니다.

우선 MP3 파일 등의 정적 파일을 Whisper로 **텍스트 변환(필사)**하는 기본 방법을 살펴보겠습니다. 예제로 간단한 영어 음성 MP3 파일을 텍스트로 변환해보겠습니다.

### 기본 사용법: 음성 파일 필사 요청
1. 오디오 파일 준비: 변환하고자 하는 음성 파일의 경로를 지정합니다. Whisper API는 여러 형식의 오디오를 지원하는데, 일반적으로 wav나 mp3를 많이 사용합니다. 여기서는 예시로 speech.mp3 파일을 사용하겠습니다.
2. API 요청 구성: OpenAI Python SDK의 client.audio.transcriptions.create 메서드를 사용하여 음성 필사 요청을 보냅니다. 주요 파라미터:
    - file: 열어둔 오디오 파일 객체 ("rb" 모드로 연 파일),
    - model: 사용할 모델 ID ("whisper-1"로 지정하여 Whisper 모델 사용).

    기본적으로 Whisper API는 응답으로 JSON 형식 데이터를 반환하며, 그 안에 "text" 필드로 변환된 텍스트를 제공합니다. response_format 파라미터를 사용해 응답 형식을 변경할 수 있는데, 기본값 "json" 외에 "text", "srt" 등 다양한 옵션이 있습니다. 우선 이해를 돕기 위해 응답을 순수 텍스트로 받도록 response_format="text"를 지정해보겠습니다.

3. 응답 처리: Whisper API는 음성 내용을 텍스트로 변환하여 반환합니다. response_format="text"로 요청한 경우 변환된 텍스트 문자열을 직접 반환하므로, 이를 출력하거나 변수에 저장할 수 있습니다.

다음은 MP3 파일을 읽어서 Whisper로 전송하고, 결과 텍스트를 출력하는 기본 코드 예제입니다:


In [ ]:
# 1. 음성 파일 열기
audio_file = open('data/speech.mp3', 'rb')
# 2. whisper API 음성 필사 요청
response = client.audio.transcriptions.create(
    file = audio_file,
    model = "whisper-1",
    response_format="text"
)
# 3. 필사된 text 출력
print(response)
audio_file.close()

In [3]:
with open('data/speech.mp3', 'rb') as audio_file:
    response = client.audio.transcriptions.create(
        file = audio_file,
        model = "whisper-1",
        response_format="text"
    )
print(response)

OpenAI Whisper is an advanced speech recognition model.



위 코드가 실행되면, speech.mp3 파일의 음성이 텍스트로 변환되어 response에 담기고, 그 내용을 출력합니다. 예를 들어 입력 음성이 영어로 "OpenAI Whisper is an advanced speech recognition model." 라는 문장을 담고 있었다면, 출력은 다음과 같은 텍스트가 될 것입니다:
```
OpenAI Whisper is an advanced speech recognition model.
```
이제 Whisper API의 기본 동작을 확인했으므로, 추가적인 기능들과 상황별 활용법을 알아보겠습니다.

## 3. 다양한 언어의 음성 인식 및 언어감지

Whisper 모델은 다국어 음성 인식을 지원합니다. 90개가 넘는 언어로 훈련되어 다양한 언어의 음성을 텍스트로 변환할 수 있습니다. 기본적으로 API에 언어를 명시하지 않으면 Whisper가 음성의 언어를 자동으로 감지하여 해당 언어로 텍스트를 필사합니다. 예를 들어, 한국어 음성을 입력하면 자동으로 한국어로 텍스트를 출력합니다. 

언어 감지 및 지정 (language 파라미터):

자동 감지가 대부분 잘 동작하지만, 경우에 따라 잘못된 언어로 인식될 가능성도 있습니다. Whisper API는 이런 경우를 대비해 language 파라미터로 입력 음성의 언어를 직접 지정하는 기능을 제공합니다. language 파라미터에 ISO 언어 코드를 설정하면 모델이 해당 언어로 인식하도록 강제할 수 있습니다.
- 예를 들어, 한국어 음성 파일을 확실히 한국어로 인식시키고 싶다면 language="ko"로 지정할 수 있습니다.
- 영어라면 language="en", 스페인어라면 language="es" 등으로 언어 코드를 설정합니다.

다음은 스페인어 음성 파일을 Whisper로 변환하는 코드 예시입니다. Whisper가 자동으로 언어를 감지하게 할 수도 있지만, 예시에서는 language="es"를 명시적으로 지정해보겠습니다:


In [4]:
# 스페인어 필사
with open('data/spanish_audio.mp3', "rb") as audio_file:
    response = client.audio.transcriptions.create(
        file=audio_file,
        model = 'whisper-1',
        response_format="text",
        language="es" # 음성 언어를 지정
    )
print(response)

¿Qué crees que es la inteligencia artificial?



In [6]:
# 음성을 번역(영어번역만 가능)
with open('data/spanish_audio.mp3', "rb") as audio_file:
    response = client.audio.translations.create(
        file=audio_file,
        model = 'whisper-1',
        response_format="text"
    )
print(response)

What do you think artificial intelligence is?



In [7]:
# 음성을 한국어로 번역(2단계) : 음성->스페인어필사->한국어
with open('data/spanish_audio.mp3', 'rb') as audio_file:
    transcription = client.audio.transcriptions.create(
        file  = audio_file,
        model = 'whisper-1',
        response_format='text',
        language='es'
    )
# 스페인어를 한국어번역
response = client.chat.completions.create(
    model = 'gpt-4.1-nano',
    messages= [{
        'role':'user', 
        'content' : f'다음 스페인어를 한국어로 번역해 주세요 : {transcription}'
    }]
)

In [9]:
response.choices[0].message.content

'인공지능이 무엇이라고 생각하세요?'

In [10]:
# 한국어 필사
with open('data/output_ko_onyx.mp3', 'rb') as audio_file:
    transcription_ko = client.audio.transcriptions.create(
        model = 'whisper-1',
        file  = audio_file,
        response_format='text',
        language='ko'
    )
print(transcription_ko)

안녕하세요. 저는 OpenAPI로 생성된 한국어 음성입니다. 한국어로 자동인식되어 읽고 있습니다.

